# Import libraries and model

In [1]:
from os import listdir
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import gc
import gensim.downloader as api

import torch
import torch.nn as nn

from math import ceil
import torch.optim as optim

nltk.download('stopwords')
nltk.download('wordnet')

# Load GloVe model with Gensim's API - Twitter specific embedding
embeddings_model = api.load("glove-twitter-200")  # 200-dimensional GloVe embeddings

#To check that T4 GPU is connected
#!nvidia-smi

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/infres/kbrowder-24/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/infres/kbrowder-24/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# %pip install "modin[all]"
# import modin.pandas as pd

In [3]:
import pandas as pd

In [4]:
# %pip install swifter
# import swifter

In [5]:
import swifter

/home/infres/kbrowder-24/DeepLearningChallengeCSC_51054_EP/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# import os
# os.environ["MODIN_ENGINE"] = "dask"  # Options: "ray" or "dask"
# import modin.pandas as pd

In [7]:
# swifter

# Data preprocessing

In [8]:
# Read all training files and concatenate them into one dataframe

#import os
#print(os.getcwd())

li = []
i = 0
for filename in listdir("train_tweets"):
    if filename != '.ipynb_checkpoints':
        print(filename)
        df = pd.read_csv("train_tweets/" + filename)
        # df.drop(columns=['Timestamp'], inplace=True)
        # drop unused column(s)
        df['MatchID'] = str(i)
        df['ID'] = str(i)+ '_' + df['PeriodID'].astype(str)
        # makes sure that the match IDs are ordered from 0,1,2... with no missing values
        i+=1
        li.append(df)
df = pd.concat(li, ignore_index=True)
#print(len(df))
df

USASlovenia2010.csv
ArgentinaBelgium72.csv
AustraliaSpain34.csv
ArgentinaGermanyFinal77.csv
AustraliaNetherlands29.csv
BelgiumSouthKorea59.csv
HondurasSwitzerland54.csv
FranceGermany70.csv
GermanyBrazil74.csv
GermanyUSA57.csv
MexicoCroatia37.csv
FranceNigeria66.csv
CameroonBrazil36.csv
NetherlandsChile35.csv
PortugalGhana58.csv
GermanyAlgeria67.csv


,ID,MatchID,PeriodID,EventType,Timestamp,Tweet
0,0_0,0,0,0,1276869000000,#USA All My Stateside Followers Stand Up And R...
1,0_0,0,0,0,1276869000000,@Lynz_89 I think the ref might have been Basil...
2,0_0,0,0,0,1276869000000,Hoping a #USA win can help ease the pain of la...
3,0_0,0,0,0,1276869000000,When does this actually start? #worldcup
4,0_0,0,0,0,1276869000000,Hanson and Roy are a proper pundit line up. #w...
...,...,...,...,...,...,...
5056045,15_169,15,169,0,1404168000000,RT @FOXSoccer: 3/4 of the #WorldCup quarterfin...
5056046,15_169,15,169,0,1404168000000,RT @Rodolph_hilal: Plz guys RETWEET .. \n\nLet...
5056047,15_169,15,169,0,1404168000000,RT @Joey7Barton: Algeria can take a lot of pos...
5056048,15_169,15,169,0,1404168000000,"RT @caughtoffside: #ALG gave it their all, was..."


In [9]:
del li

In [10]:
gc.collect()

0

In [11]:
# Preprocessing of tweet
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Remove non letter and whitespace
    text = re.sub(r'[^a-z\s]', '', text)
    # Remove numbers
    # text = re.sub(r'\d+', '', text)
    # Tokenization
    words = text.split()
    # Remove stopwords and lemmatize
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    # Lemmatization
    # words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

In [12]:
# Apply preprocessing to each tweet
df['Tweet'] = df['Tweet'].swifter.apply(preprocess_text)
df

Pandas Apply: 100%|██████████| 5056050/5056050 [02:55<00:00, 28851.70it/s]


,ID,MatchID,PeriodID,EventType,Timestamp,Tweet
0,0_0,0,0,0,1276869000000,usa stateside follower stand represent beautif...
1,0_0,0,0,0,1276869000000,lynz think ref might basil fawlty actually wor...
2,0_0,0,0,0,1276869000000,hoping usa win help ease pain last night loss ...
3,0_0,0,0,0,1276869000000,actually start worldcup
4,0_0,0,0,0,1276869000000,hanson roy proper pundit line worldcup
...,...,...,...,...,...,...
5056045,15_169,15,169,0,1404168000000,rt foxsoccer worldcup quarterfinal set ger v f...
5056046,15_169,15,169,0,1404168000000,rt rodolphhilal plz guy retweet let trend alg ...
5056047,15_169,15,169,0,1404168000000,rt joeybarton algeria take lot positive big bi...
5056048,15_169,15,169,0,1404168000000,rt caughtoffside alg gave wasnt enough heart t...


# Tweet Embeddings

In [13]:
# Get vector tweet embeddings
# TODOOOOOOOOOOOOOOOO maybe instead of avg word embedding for each tweet can get sentence
#   embeddings to retain more information
#   -> can try more complex functions here
#   -> avg embedding of each word for a tweet is fine for now, maybe works well enough

# Function to compute the average word vector for a tweet
def get_avg_embedding(tweet, model, vector_size=200):
    words = tweet.split()  # Tokenize by whitespace
    word_vectors = [model[word] for word in words if word in model]
    if not word_vectors:  # If no words in the tweet are in the vocabulary, return a zero vector
        return np.zeros(vector_size)
    return np.mean(word_vectors, axis=0)

In [14]:
# Crashes after using all available RAM :( on google colab
# 

# obtain vector tweet embeddings
vector_size = 200  # Adjust based on the chosen GloVe model
# tweet_vectors = np.vstack([get_avg_embedding(tweet, embeddings_model, vector_size) for tweet in df['Tweet']])
# tweet_df = pd.DataFrame(tweet_vectors)
# tweet_df.head()

In [15]:
from functools import partial

f = partial(get_avg_embedding, model=embeddings_model,vector_size=vector_size)
f.__name__ = "paritla"
f.__module__ = get_avg_embedding.__module__

df["tweet_vector"] = df["Tweet"].swifter.apply(f)

Pandas Apply: 100%|██████████| 5056050/5056050 [02:23<00:00, 35148.88it/s]


In [16]:
# Attach the vectors into the original dataframe
# df = pd.concat([df, tweet_df], axis=1)

# Drop the columns that are not useful anymore
# no need for Tweet column since we have its corresponding vector embedding
df.drop(columns=['Tweet'], inplace=True)



In [17]:
df.head()

,ID,MatchID,PeriodID,EventType,Timestamp,tweet_vector
0,0_0,0,0,0,1276869000000,"[0.066179484, 0.25621355, 0.08082778, -0.33907..."
1,0_0,0,0,0,1276869000000,"[-0.1474687, 0.3667189, 0.10468656, 0.049433, ..."
2,0_0,0,0,0,1276869000000,"[0.22338197, 0.27252772, 0.07159816, -0.293135..."
3,0_0,0,0,0,1276869000000,"[-0.10895123, 0.28830335, 0.40979335, -0.22218..."
4,0_0,0,0,0,1276869000000,"[-0.112306446, 0.11411217, 0.24610366, -0.4545..."


In [18]:
# by now should have df with columns: ID, match id, period id, Event Type, tweet_vector. Tweet_vector is just 200 columns
df

,ID,MatchID,PeriodID,EventType,Timestamp,tweet_vector
0,0_0,0,0,0,1276869000000,"[0.066179484, 0.25621355, 0.08082778, -0.33907..."
1,0_0,0,0,0,1276869000000,"[-0.1474687, 0.3667189, 0.10468656, 0.049433, ..."
2,0_0,0,0,0,1276869000000,"[0.22338197, 0.27252772, 0.07159816, -0.293135..."
3,0_0,0,0,0,1276869000000,"[-0.10895123, 0.28830335, 0.40979335, -0.22218..."
4,0_0,0,0,0,1276869000000,"[-0.112306446, 0.11411217, 0.24610366, -0.4545..."
...,...,...,...,...,...,...
5056045,15_169,15,169,0,1404168000000,"[0.13047262, 0.2851209, 0.018671205, -0.052115..."
5056046,15_169,15,169,0,1404168000000,"[0.30743918, 0.36894467, -0.1729, -0.27685714,..."
5056047,15_169,15,169,0,1404168000000,"[0.14770256, 0.20187803, -0.1268579, -0.095563..."
5056048,15_169,15,169,0,1404168000000,"[0.016947214, 0.28408444, 0.08043686, 0.101969..."


In [ ]:
period_chunks = 10

In [20]:
# Sort by MatchID, PeriodID, and Timestamp to maintain order
df = df.sort_values(by=['MatchID', 'PeriodID', 'Timestamp']).reset_index(drop=True)

# Helper function to assign chunks
def assign_chunks(group, n_chunks=period_chunks):
    chunk_size = len(group) / n_chunks
    return (np.floor(np.arange(len(group)) / chunk_size)).astype(int)

# Apply chunk assignment within each MatchID and PeriodID
df['chunk'] = df.groupby(['MatchID', 'PeriodID']).apply(
    lambda group: assign_chunks(group)
).explode().reset_index(drop=True)

/tmp/ipykernel_2726494/1731830507.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['chunk'] = df.groupby(['MatchID', 'PeriodID']).apply(


In [21]:
# group the tweets into their corresponding periods to generate an average embedding vector for each period
# so there are no duplicate period id rows per match
# decreases size of data + makes it easier to fit into LSTM model
df = df.groupby(['MatchID', 'PeriodID', 'ID','chunk']).mean().reset_index()
df.drop(columns=['ID'], inplace=True) 
df['MatchID'] = df['MatchID'].astype(int)
df['PeriodID'] = df['PeriodID'].astype(int)
# need to convert to int before sorting
df.sort_values(by=['MatchID', 'PeriodID', 'chunk'], inplace=True)
df.reset_index(drop=True, inplace=True)


In [22]:
# df.loc[df["EventType"] != 0, "EventType"] = 1
df

,MatchID,PeriodID,chunk,EventType,Timestamp,tweet_vector
0,0,0,0,0.0,1.276869e+12,"[0.049786884, 0.1913198, 0.068040654, -0.15670..."
1,0,0,1,0.0,1.276869e+12,"[0.09133990850442877, 0.25186495755154353, 0.1..."
2,0,0,2,0.0,1.276869e+12,"[0.11688901, 0.22027843, 0.10215302, -0.159173..."
3,0,0,3,0.0,1.276869e+12,"[0.12811066, 0.23626624, 0.11524634, -0.131818..."
4,0,0,4,0.0,1.276869e+12,"[0.07077107, 0.23463808, 0.07358122, -0.123566..."
...,...,...,...,...,...,...
10680,15,169,0,0.0,1.404168e+12,"[0.12749639, 0.23482081, 0.052804522, -0.06213..."
10681,15,169,1,0.0,1.404168e+12,"[0.13463659662257052, 0.23993844279640894, 0.0..."
10682,15,169,2,0.0,1.404168e+12,"[0.12705522455795054, 0.24554795088920525, 0.0..."
10683,15,169,3,0.0,1.404168e+12,"[0.121888824, 0.24418654, 0.042192977, -0.0525..."


In [23]:
# df['tweet_vector'] = df['tweet_vector'].swifter.apply(lambda v: v / np.linalg.norm(v))

In [24]:
df_agg = (
    df.groupby(['MatchID', 'PeriodID'])
    .agg(
        period_matrix=('tweet_vector', lambda x: np.stack(x.to_numpy())),  # Stack the mean vectors
        mean_event_type=('EventType', 'mean')  # Compute the mean event type
    )
    .reset_index()
)

In [25]:
df = df_agg

In [26]:
df.rename(columns={'period_matrix': 'tweet_vector', 'mean_event_type':'EventType'}, inplace=True)

In [27]:
df

,MatchID,PeriodID,tweet_vector,EventType
0,0,0,"[[0.04978688433766365, 0.1913197934627533, 0.0...",0.0
1,0,1,"[[0.09943458437919617, 0.1979677975177765, 0.1...",0.0
2,0,2,"[[0.1325032, 0.21106924, 0.112774804, -0.15817...",0.0
3,0,3,"[[0.11102492362260818, 0.21936477720737457, 0....",1.0
4,0,4,"[[0.1170713551396477, 0.20770213977042315, 0.1...",0.0
...,...,...,...,...
2132,15,165,"[[0.12381656467914581, 0.24038036167621613, 0....",1.0
2133,15,166,"[[0.13367936, 0.2413072, 0.04921915, -0.065234...",1.0
2134,15,167,"[[0.12520525245914096, 0.23613955238201428, 0....",1.0
2135,15,168,"[[0.13684346, 0.24208261, 0.042130414, -0.0524...",1.0


# Separate Train and Test data

In [28]:
# train on of the first 13 of 16 matches (16*0.8=12.8~=13)
# and the test data would be the last 3 matches. 
# Before submitting on Kaggle we should train on full dataset, so al 16 matches
train_percentage = 0.8
unique_match_ids = df['MatchID'].unique()
print(unique_match_ids)
num_matches_training = int(ceil(len(unique_match_ids)*train_percentage))
print(num_matches_training)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
13


In [29]:
target_match_id = num_matches_training
# target_match_id is first match id that will appear in test set
# all matches from target_match_id and after will be in test test
print(target_match_id)

13


In [30]:

#df2 = df['MatchID'] == 15
#df2

In [31]:
# row_index is first row with match id target_match_id
# row_index is then the first row of the matches that will go to the test


row_index = (df['MatchID'] == target_match_id).idxmax()
#row_index = df[df['MatchID'] == target_match_id].first_valid_index()
df_X_train = df[:row_index].copy()
df_X_test = df[row_index:].copy()


In [32]:
df_y_train = df_X_train['EventType']
df_y_test = df_X_test['EventType']

In [33]:
df_y_train

0       0.0
1       0.0
2       0.0
3       1.0
4       0.0
       ... 
1702    1.0
1703    1.0
1704    1.0
1705    1.0
1706    1.0
Name: EventType, Length: 1707, dtype: float64

In [34]:
df_y_test.reset_index(drop=True, inplace=True)
df_y_test

0      0.0
1      0.0
2      1.0
3      1.0
4      1.0
      ... 
425    1.0
426    1.0
427    1.0
428    1.0
429    0.0
Name: EventType, Length: 430, dtype: float64

In [35]:
df_X_train.drop(['EventType'], axis=1, inplace=True)
df_X_test.drop(['EventType'], axis=1, inplace=True)

In [36]:
df_X_train

,MatchID,PeriodID,tweet_vector
0,0,0,"[[0.04978688433766365, 0.1913197934627533, 0.0..."
1,0,1,"[[0.09943458437919617, 0.1979677975177765, 0.1..."
2,0,2,"[[0.1325032, 0.21106924, 0.112774804, -0.15817..."
3,0,3,"[[0.11102492362260818, 0.21936477720737457, 0...."
4,0,4,"[[0.1170713551396477, 0.20770213977042315, 0.1..."
...,...,...,...
1702,12,125,"[[0.01331692, 0.21187948, 0.1284352, -0.189622..."
1703,12,126,"[[0.016792007, 0.19603887, 0.12457686, -0.1841..."
1704,12,127,"[[0.008126191, 0.23228367, 0.10223757, -0.2090..."
1705,12,128,"[[0.009118693, 0.22613892, 0.106851995, -0.202..."


In [37]:
df_X_test.reset_index(drop=True, inplace=True)
df_X_test

,MatchID,PeriodID,tweet_vector
0,13,0,"[[0.15184887, 0.2528863, -0.03571834, -0.08834..."
1,13,1,"[[0.16123801, 0.25753284, -0.015193953, -0.090..."
2,13,2,"[[0.14436208, 0.25661135, -0.016834391, -0.091..."
3,13,3,"[[0.15383919, 0.21019702, 0.0036868062, -0.104..."
4,13,4,"[[0.1350092, 0.21672635, -0.012920625, -0.1162..."
...,...,...,...
425,15,165,"[[0.12381656467914581, 0.24038036167621613, 0...."
426,15,166,"[[0.13367936, 0.2413072, 0.04921915, -0.065234..."
427,15,167,"[[0.12520525245914096, 0.23613955238201428, 0...."
428,15,168,"[[0.13684346, 0.24208261, 0.042130414, -0.0524..."


In [38]:
# now df_X_train and df_X_test should have columns MatchID, PeriodID, tweet_vector. Tweet_vector is just 200 columns
# df_y_train and df_y_test should have 1 column, EventType
# the matchids are grouped together so all the rows of the same
# match ids are grouped next to each other, and the periodID are ordered chronologically.

In [39]:
# not working on my machine so I keep DF

# now we have df_X_train, df_X_test, df_y_train, df_y_test
# we no longer need df so we should free up the memory
# del df  # remove reference to the original DataFrame
# gc.collect()  # force garbage collection to free up memory

In [40]:
max_periods = df_X_train.groupby('MatchID')['PeriodID'].max().reset_index()
max_periods
# as we can see not every match has the same number of periods!

,MatchID,PeriodID
0,0,129
1,1,129
2,2,129
3,3,179
4,4,96
5,5,129
6,6,129
7,7,129
8,8,129
9,9,129


In [41]:
max_periods = df_X_test.groupby('MatchID')['PeriodID'].max().reset_index()
max_periods

,MatchID,PeriodID
0,13,129
1,14,129
2,15,169


# Format data for PyTorch LSTM

In [42]:
# input tensor for a PyTorch LSTM should have the shape of (when setting batch_first=True)
# (batch_size, seq_len, num_features) when using the batch_first=True parameter
# batch_size is number of sequences processed at once

# TRY WITHOUT SLIDING WINDOW APPROACH
#    which would mean batch size = number of matches
#    much easier to format for LSTM as 3D tensor
#    dimension of 3D tensor with batch_first=True:(batch_size = num_matches, seq_len = num_periods, num _features = 200)
#    (match_id, period_id, num_features=200)
#     not every match has the same number of periods!, so seq_len can vary between different matches
#     fix: will have to pad with zeroes
# we want tensor[match_id][period_id] to return list len 200 of corresponding tweet vector


In [43]:
df_X_train

,MatchID,PeriodID,tweet_vector
0,0,0,"[[0.04978688433766365, 0.1913197934627533, 0.0..."
1,0,1,"[[0.09943458437919617, 0.1979677975177765, 0.1..."
2,0,2,"[[0.1325032, 0.21106924, 0.112774804, -0.15817..."
3,0,3,"[[0.11102492362260818, 0.21936477720737457, 0...."
4,0,4,"[[0.1170713551396477, 0.20770213977042315, 0.1..."
...,...,...,...
1702,12,125,"[[0.01331692, 0.21187948, 0.1284352, -0.189622..."
1703,12,126,"[[0.016792007, 0.19603887, 0.12457686, -0.1841..."
1704,12,127,"[[0.008126191, 0.23228367, 0.10223757, -0.2090..."
1705,12,128,"[[0.009118693, 0.22613892, 0.106851995, -0.202..."


In [44]:
# modified for array tweet_vector column
def convert_df_to_3D_tensor(df_X, df_y):
    # df_X should have columns MatchID, PeriodID, tweet_vector. Tweet_vector is just 200x1 array
    # rows with same matchID should be grouped together (adjacent rows)
    # df_y should have one column (the EventType)
    # returns tensor_X numpy array already padded! shape: (num_matches, max_num_periods, num _features = 200)
    # and tensor_y of shape: (num_matches, max_num_periods)
    num_matches = len(df_X['MatchID'].unique())
    max_periods = df_X.groupby('MatchID')['PeriodID'].max().reset_index()
    total_max_period = max_periods['PeriodID'].max()
    #total_max_period is max seq len

    tensor_X = np.zeros((len(df_X), period_chunks, 200))

    tensor_y = np.zeros((len(df_y,)))
    print(tensor_X.shape)
    print(tensor_y.shape)
    
    for row_index, row in df_X.iterrows():
        i = row_index        
        features = row['tweet_vector']  # Skip MatchID and PeriodID
        tensor_X[i, :, :] = features
        tensor_y[i] = df_y[row_index]
        
    return tensor_X, tensor_y


X_train_tensor, y_train_tensor = convert_df_to_3D_tensor(df_X_train, df_y_train)
# X_train_tensor[match_id][period_id] to return list len 200 of corresponding tweet vector
# y_train_tensor[match_id][period_id] to return corresponding EventType (1 or 0)
# match_id index starts at 0 even if first match in df doesnt have match id 0
#X_train_tensor[12][175]
#X_train_tensor[12][179]
#X_train_tensor[2][129]


(1707, 5, 200)
(1707,)


In [45]:
#print(X_train_tensor[0][3])
#print(y_train_tensor[0][3])

In [46]:
# SCALE DATA? minmaxscaler for example!
# SCALING MIGHT BE UNNECESSARY SINCE OUTPUT OF GLOVE TWEET 200 IS ALREADY SCALED BETWEEN -1 AND 1
#scaler = MinMaxScaler()
#tensor = scaler.fit_transform(tensor)

# CONVERT TO PYTORCH TENSOR
X_train_tensor = torch.tensor(X_train_tensor, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_tensor, dtype=torch.float32)

print(X_train_tensor.shape)
print(y_train_tensor.shape)
# X_train_tensor, y_train_tensor are now pytorch tensors

torch.Size([1707, 5, 200])
torch.Size([1707])


# LSTM Model

In [47]:
# TODO VERIFY ITS CORRECT + MAKE MORE SOPHISTICATED
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout_rate):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_rate)
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out)
        out = out[:,-1,:]
        out = self.sigmoid(out) # applying sigmoid to convert to probabilities
        return out.squeeze(-1)

#TODOOOOOOOOOO torch.nn.utils.rnn.pack_padded_sequence. This allows the model to ignore the padded values during computation.

In [112]:
# Hyperparams
batch_size = 120
hidden_size = 200 # can tune
num_layers = 5 # can tune
dropout_rate = 0.7 # can tune
num_epochs = 800 # can tune
lr = 0.0003 # can tune

In [113]:
batch_idxs = list(range(0,X_train_tensor.shape[0],batch_size))
batched_data = []
for idx in batch_idxs:
    batched_data.append((X_train_tensor[idx:idx+batch_size], y_train_tensor[idx:idx+batch_size]))

In [114]:
gpu = torch.device('cuda:1')

In [115]:
# convert df_X_test and df_y_test to correct format/dimensions
X_test_tensor, y_test_tensor = convert_df_to_3D_tensor(df_X_test, df_y_test)
# CONVERT TO PYTORCH TENSOR
X_test_tensor = torch.tensor(X_test_tensor, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_tensor, dtype=torch.float32)

(430, 5, 200)
(430,)


# Train model

In [116]:
model = LSTMModel(input_size=200, hidden_size=hidden_size, num_layers=num_layers, dropout_rate=dropout_rate)
model = model.to(gpu)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCELoss() # great for binary classification
criterion = criterion.to(gpu)
#print(f"Shape of X_train_tensor: {X_train_tensor.shape}")
model.train()
for epoch in range(num_epochs):
    optimizer.zero_grad()
    for train, label in batched_data:
        train = train.to(gpu)
        label = label.to(gpu)
        outputs = model(train)
        #print(f"shape of outputs: {outputs.shape}")
        
        loss = criterion(outputs, label)
        loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        predict = model(X_test_tensor.to(gpu))
        e_loss = criterion(predict, y_test_tensor.to(gpu))
        print(f"Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}", f"Eval Loss: {e_loss.item():.4f}")

print("Model is trained! (on training data)")

Epoch [0/800], Loss: 0.6958 Eval Loss: 0.6933
Epoch [10/800], Loss: 0.6496 Eval Loss: 0.7048
Epoch [20/800], Loss: 0.6106 Eval Loss: 0.7197
Epoch [30/800], Loss: 0.6249 Eval Loss: 0.7100
Epoch [40/800], Loss: 0.5956 Eval Loss: 0.7157
Epoch [50/800], Loss: 0.4969 Eval Loss: 0.6898
Epoch [60/800], Loss: 0.5776 Eval Loss: 0.6450
Epoch [70/800], Loss: 0.4781 Eval Loss: 0.6184
Epoch [80/800], Loss: 0.4892 Eval Loss: 0.5887
Epoch [90/800], Loss: 0.5078 Eval Loss: 0.5920
Epoch [100/800], Loss: 0.5210 Eval Loss: 0.5897
Epoch [110/800], Loss: 0.5327 Eval Loss: 0.5950
Epoch [120/800], Loss: 0.5194 Eval Loss: 0.5972
Epoch [130/800], Loss: 0.5329 Eval Loss: 0.5828
Epoch [140/800], Loss: 0.5213 Eval Loss: 0.5883
Epoch [150/800], Loss: 0.5238 Eval Loss: 0.5830
Epoch [160/800], Loss: 0.4957 Eval Loss: 0.5758
Epoch [170/800], Loss: 0.5104 Eval Loss: 0.5714
Epoch [180/800], Loss: 0.5017 Eval Loss: 0.5589
Epoch [190/800], Loss: 0.5052 Eval Loss: 0.5527
Epoch [200/800], Loss: 0.5168 Eval Loss: 0.5461
Epo

# Evaluate on test data

In [117]:
#print(X_test_tensor[2][129])
#print(y_test_tensor[2][129])

In [118]:




model.eval()

with torch.no_grad():
    predictions = model(X_test_tensor.to(gpu)).to('cpu')

# predictions have values between 0 and 1 because forward pass of LSTM contains sigmoid at output
#print(predictions)

predicted_classes = (predictions > 0.5).float() # 0.5 is threshold
#this converts to same dimensional array of True or false, and .float() converts True to 1 and False to 0

#print(predicted_classes)
 

In [119]:
predicted_classes

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
        1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

In [120]:
#print(predictions.shape)

In [121]:
# performance metrics

loss = criterion(predictions, y_test_tensor) # use predictions for loss calculation

print(f"Binary Cross-Entropy Loss: {loss.item():.4f}")

def accuracy(y_true, y_pred):
    if y_true.dtype != y_pred.dtype or y_true.shape != y_pred.shape:
        raise ValueError(f"Inputs do not have same type or shape!")
    y_rand = torch.randint(0,2,y_true.shape)
    correct_predictions = (y_true == y_pred).sum().item()
    total_predictions = y_true.numel()
    accuracy = correct_predictions / total_predictions * 100
    return accuracy
accuracy = accuracy(y_test_tensor, predicted_classes)


print(f"Accuracy: {accuracy:.4f}")

#print(y_test_tensor.shape)
#print(predicted_classes.shape)


# Visualization of Actual vs Predicted Classes
# import matplotlib.pyplot as plt
# TODO COULD USE PLT TO VISUALIZE?

Binary Cross-Entropy Loss: 0.8959
Accuracy: 69.0698


# For Kaggle Submission

In [122]:
# RETRAIN MODEL ON ENTIRE TRAINING DATA AND EVALUATE EVAL TWEETS




df_X = pd.concat([df_X_train, df_X_test], ignore_index=True)
df_y = pd.concat([df_y_train, df_y_test], ignore_index=True)




In [123]:
df_X

,MatchID,PeriodID,tweet_vector
0,0,0,"[[0.04978688433766365, 0.1913197934627533, 0.0..."
1,0,1,"[[0.09943458437919617, 0.1979677975177765, 0.1..."
2,0,2,"[[0.1325032, 0.21106924, 0.112774804, -0.15817..."
3,0,3,"[[0.11102492362260818, 0.21936477720737457, 0...."
4,0,4,"[[0.1170713551396477, 0.20770213977042315, 0.1..."
...,...,...,...
2132,15,165,"[[0.12381656467914581, 0.24038036167621613, 0...."
2133,15,166,"[[0.13367936, 0.2413072, 0.04921915, -0.065234..."
2134,15,167,"[[0.12520525245914096, 0.23613955238201428, 0...."
2135,15,168,"[[0.13684346, 0.24208261, 0.042130414, -0.0524..."


In [124]:
df_y

0       0.0
1       0.0
2       0.0
3       1.0
4       0.0
       ... 
2132    1.0
2133    1.0
2134    1.0
2135    1.0
2136    0.0
Name: EventType, Length: 2137, dtype: float64

In [125]:
# convert df_X_test and df_y_test to correct format/dimensions
X_tensor, y_tensor = convert_df_to_3D_tensor(df_X, df_y)
# CONVERT TO PYTORCH TENSOR
X_tensor = torch.tensor(X_tensor, dtype=torch.float32)
y_tensor = torch.tensor(y_tensor, dtype=torch.float32)

(2137, 5, 200)
(2137,)


In [126]:
# NOTES
# HOW TO MAKE SURE THAT we:
# 1. DO NOT ignore the order of the tweets -> (LSTM)
# 2. treat each time period as RELATED to the football match they belong to -> treat each match as a sequence, train LSTM on every sequence
#                      since pytorch tensor expects multiple sequences (batches)



# for LSTM: Each input sequence should consist of tweets from a specific match, ordered by Period ID.
#   tweets of different matches are unrelated, but tweets of a same match are related sequentially (chronologically)
#   structure training data such that tweets are grouped by match id, and ordered by period id

In [127]:
batch_size = 100
batch_idxs = list(range(0,X_tensor.shape[0],batch_size))
batched_data = []
for idx in batch_idxs:
    batched_data.append((X_tensor[idx:idx+batch_size], y_tensor[idx:idx+batch_size]))

In [128]:
model = LSTMModel(input_size=200, hidden_size=hidden_size, num_layers=num_layers, dropout_rate=dropout_rate)
model = model.to(gpu)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCELoss() # great for binary classification
criterion = criterion.to(gpu)
#print(f"Shape of X_train_tensor: {X_train_tensor.shape}")
model.train()
for epoch in range(num_epochs):
    optimizer.zero_grad()
    for train, label in batched_data:
        train = train.to(gpu)
        label = label.to(gpu)
        outputs = model(train)
        #print(f"shape of outputs: {outputs.shape}")
        
        loss = criterion(outputs, label)
        loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}")

print("Model is trained! (on training data)")

Epoch [0/800], Loss: 0.6926
Epoch [10/800], Loss: 0.6953
Epoch [20/800], Loss: 0.6938
Epoch [30/800], Loss: 0.6939
Epoch [40/800], Loss: 0.6910
Epoch [50/800], Loss: 0.6579
Epoch [60/800], Loss: 0.5723
Epoch [70/800], Loss: 0.5512
Epoch [80/800], Loss: 0.4963
Epoch [90/800], Loss: 0.5642
Epoch [100/800], Loss: 0.5415
Epoch [110/800], Loss: 0.5118
Epoch [120/800], Loss: 0.4900
Epoch [130/800], Loss: 0.4821
Epoch [140/800], Loss: 0.5063
Epoch [150/800], Loss: 0.4592
Epoch [160/800], Loss: 0.4293
Epoch [170/800], Loss: 0.4218
Epoch [180/800], Loss: 0.4180
Epoch [190/800], Loss: 0.4221
Epoch [200/800], Loss: 0.4091
Epoch [210/800], Loss: 0.3991
Epoch [220/800], Loss: 0.4051
Epoch [230/800], Loss: 0.4090
Epoch [240/800], Loss: 0.3748
Epoch [250/800], Loss: 0.3957
Epoch [260/800], Loss: 0.3426
Epoch [270/800], Loss: 0.3694
Epoch [280/800], Loss: 0.3466
Epoch [290/800], Loss: 0.3577
Epoch [300/800], Loss: 0.3730
Epoch [310/800], Loss: 0.3326
Epoch [320/800], Loss: 0.3313
Epoch [330/800], Loss

In [129]:
# Read all training files and concatenate them into one dataframe

#import os
#print(os.getcwd())

li = []
i = 0
for filename in listdir("eval_tweets"):
    if filename != '.ipynb_checkpoints':
        print(filename)
        df = pd.read_csv("eval_tweets/" + filename)
        # df.drop(columns=['Timestamp'], inplace=True)
        # drop unused column(s)
        # makes sure that the match IDs are ordered from 0,1,2... with no missing values
        i+=1
        li.append(df)
eval_df = pd.concat(li, ignore_index=True)
#print(len(df))
eval_df

GreeceIvoryCoast44.csv
NetherlandsMexico64.csv


GermanyGhana32.csv
GermanySerbia2010.csv


,ID,MatchID,PeriodID,Timestamp,Tweet
0,9_0,9,0,1403639400000,Wana place a bet on an ivory coast win but ain...
1,9_0,9,0,1403639400000,#WatchLive @FIFAWorldCup: Greece vs. Ivory Coa...
2,9_0,9,0,1403639400000,Gonna watch the Colombia and Japan game becaus...
3,9_0,9,0,1403639400000,#CIV vs #COL & #GRE vs #JPN! It would be inter...
4,9_0,9,0,1403639400000,RT @DuncanCastles: Quite a statistic this: Gre...
...,...,...,...,...,...
1072923,16_129,16,129,1276867800000,LETS GO #USA #worldcup
1072924,16_129,16,129,1276867800000,another upset in #WC2010 #Srb beat #Ger by 1-0 !!
1072925,16_129,16,129,1276867800000,RT @FIFAcom: #GER 0:1 #SRB: TheÂ finalÂ whistl...
1072926,16_129,16,129,1276867800000,dukung yg menang -_- RT @AlikaZahira: #bra #fr...


In [130]:
# Apply preprocessing to each tweet
eval_df['Tweet'] = eval_df['Tweet'].swifter.apply(preprocess_text)
eval_df

Pandas Apply: 100%|██████████| 1072928/1072928 [00:37<00:00, 28824.32it/s]


,ID,MatchID,PeriodID,Timestamp,Tweet
0,9_0,9,0,1403639400000,wana place bet ivory coast win aint sure sugge...
1,9_0,9,0,1403639400000,watchlive fifaworldcup greece v ivory coast ta...
2,9_0,9,0,1403639400000,gonna watch colombia japan game watching ivory...
3,9_0,9,0,1403639400000,civ v col gre v jpn would interesting greece w...
4,9_0,9,0,1403639400000,rt duncancastles quite statistic greece never ...
...,...,...,...,...,...
1072923,16_129,16,129,1276867800000,let go usa worldcup
1072924,16_129,16,129,1276867800000,another upset wc srb beat ger
1072925,16_129,16,129,1276867800000,rt fifacom ger srb final whistle sound httpbit...
1072926,16_129,16,129,1276867800000,dukung yg menang rt alikazahira bra fra arg wo...


In [131]:
eval_df["tweet_vector"] = eval_df["Tweet"].swifter.apply(f)

Pandas Apply: 100%|██████████| 1072928/1072928 [00:30<00:00, 35129.03it/s]


In [132]:
# Sort by MatchID, PeriodID, and Timestamp to maintain order
eval_df = eval_df.sort_values(by=['MatchID', 'PeriodID', 'Timestamp']).reset_index(drop=True)

# Helper function to assign chunks
def assign_chunks(group, n_chunks=10):
    chunk_size = len(group) / n_chunks
    return (np.floor(np.arange(len(group)) / chunk_size)).astype(int)

# Apply chunk assignment within each MatchID and PeriodID
eval_df['chunk'] = eval_df.groupby(['MatchID', 'PeriodID']).apply(
    lambda group: assign_chunks(group)
).explode().reset_index(drop=True)

/tmp/ipykernel_2726494/3165773019.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  eval_df['chunk'] = eval_df.groupby(['MatchID', 'PeriodID']).apply(


In [133]:
eval_df.drop(columns=['Tweet'], inplace=True)

In [134]:
# group the tweets into their corresponding periods to generate an average embedding vector for each period
# so there are no duplicate period id rows per match
# decreases size of data + makes it easier to fit into LSTM model
eval_df = eval_df.groupby(['MatchID', 'PeriodID', 'ID','chunk']).mean().reset_index()
eval_df.drop(columns=['ID'], inplace=True) 
eval_df['MatchID'] = eval_df['MatchID'].astype(int)
eval_df['PeriodID'] = eval_df['PeriodID'].astype(int)
# need to convert to int before sorting
eval_df.sort_values(by=['MatchID', 'PeriodID', 'chunk'], inplace=True)
eval_df.reset_index(drop=True, inplace=True)

In [135]:
eval_df_agg = (
    eval_df.groupby(['MatchID', 'PeriodID'])
    .agg(
        period_matrix=('tweet_vector', lambda x: np.stack(x.to_numpy())),  # Stack the mean vectors
    )
    .reset_index()
)

In [136]:
eval_df = eval_df_agg

In [137]:
eval_df.rename(columns={'period_matrix': 'tweet_vector'}, inplace=True)

In [138]:
eval_df

,MatchID,PeriodID,tweet_vector
0,6,0,"[[0.16653539, 0.26003703, 0.059985828, -0.0924..."
1,6,1,"[[0.15020987, 0.2654425, 0.05469284, -0.114660..."
2,6,2,"[[0.16435114, 0.27378768, 0.05723621, -0.10151..."
3,6,3,"[[0.17175293896567598, 0.27191852900369834, 0...."
4,6,4,"[[0.15491624, 0.28381658, 0.058633655, -0.0947..."
...,...,...,...
511,16,125,"[[0.09658312052488327, 0.2646336853504181, 0.1..."
512,16,126,"[[0.07132586327974091, 0.2369798426262357, 0.0..."
513,16,127,"[[0.08168929815292358, 0.2357352077960968, 0.1..."
514,16,128,"[[0.13000393, 0.25957957, 0.109353416, -0.1556..."


In [139]:
eval_X = np.zeros((len(eval_df), 10, 200))

for row_index, row in eval_df.iterrows():
    i = row_index        
    features = row['tweet_vector']  # Skip MatchID and PeriodID
    eval_X[i, :, :] = features

In [140]:
eval_X = torch.tensor(eval_X, dtype=torch.float32)

In [141]:
model.eval()
eval_predictions = model(eval_X.to(gpu)).to('cpu')

In [142]:
eval_predicted_classes = (eval_predictions > 0.5).float()

In [143]:
import datetime

eval_df['ID'] = eval_df.apply(lambda row: str(row.MatchID) + '_' + str(row.PeriodID), axis=1)
eval_df['EventType'] = [float(x) for x in eval_predicted_classes]
prediction_df = eval_df[['ID','EventType']]


submission_filename = "submissions/Submission " + datetime.datetime.now().strftime("%d%m-%H%M%S") + ".csv"
prediction_df.to_csv(submission_filename, index=False)